# Gathering Merger Trees of the Most Massive Galaxies

In this document, we'll navigate the IllustrisTNG Project Database to identify the most massive galaxies in the TNG100-1 simulation and obtain Merger Trees.

As shown in the document **Data Structure of IllustrisTNG Database**, the .png file of the merger tree is available. Let's navigate there. 

First, load all the required packages and important data:

In [2]:
import requests                 # To use the helper function "get"
import h5py                     # To read .hdf5 files in Python
import matplotlib.pyplot as mpl # To plot using the data from .hdf5 fiels
from skimage import io          # To work with images
import numpy as np              # To calculate big numerical data
import cv2                      # To create the video
import os                       # To navigate working directories

baseUrl = 'http://www.tng-project.org/api/'               # URL to the API Root
headers = {"api-key":"your_key_here"}                     # API Key required to access the database
                                                          # Make sure to change this!

We also need the helper function we used in the other document to naviage the API:

In [3]:
def get(path, params=None): # This is provided by the IllustrisTNG Project
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

Since all the details on the navigation and data structure are already explained in the other document, we'll navigate straight to our point of interest, *Subhalo List* page at $z=0$.

In [6]:
root = get(baseUrl)                                   # Getting to API Root
names = [sim['name'] for sim in root['simulations']]  # Creating list of simulations
i = names.index('TNG100-1')                           # Finding where 'TNG100-1' is stored in the list
sim = get(root['simulations'][i]['url'])              # Navigating to Simulation Details page
snaps = get(sim['snapshots'])                         # Navigating to Snapshot List page
snap = get(snaps[-1]['url'])                          # Naviagting to Snapshot Details page for z=0
subs = get(snap['subhalos'],                          
           {'limit':20, 'order_by':'-mass_stars'})    # Navigating to Subhalo List page for z=0,
                                                      #     ordered in decreasing mass. 

Let's get top 5 listing of subhalos, in order of decreasing mass.

In [11]:
ids = [subs['results'][i]['id'] for i in range(5) ]
ids

[0, 31342, 17185, 69507, 83280]

To make the organization easier, let's create a directory for these subhalo IDs.

In [13]:
parent_dir = '/Users/hwoo1211/Jupyter Notebook/IllustrisTNG Project/Merger Trees'
for id in ids:
    path = os.path.join(parent_dir, str(id))
    os.mkdir(path)

Then, change the working directory to the first folder:

In [21]:
img_dir = parent_dir + '/' + str(ids[0])
os.chdir(img_dir)
os.getcwd()

'/Users/hwoo1211/Jupyter Notebook/IllustrisTNG Project/Merger Trees/0'

Now let's get the details on this specific subhalo and grab the simple SubLink Tree data:

In [16]:
sub = get(subs['results'][0]['url'])
simple = get(sub['trees']['sublink_simple'])
len(simple['Main'])

99

From here, we can sort the tree data *simple* by increasing snapshot number:

In [17]:
sortSimple = sorted(simple['Main'])
len(sortSimple)

99

Fetch the .png pictures of the Merge Tree (This will take a long time, since it's obtaining the *len(sortSimple)* amount of sublink trees):

In [19]:
images = []

for i in range(len(simple['Main'])):
    url = 'https://www.tng-project.org/api/TNG100-1/snapshots/'+ str(sortSimple[i][0])+'/subhalos/' + str(sortSimple[i][1])+ '/'
    subhalo = get(url)
    image = get(subhalo['vis']['mergertree_sublink'])
    images.append(image)

And create the video:

In [22]:
imgs = [img for img in os.listdir(img_dir) if img.endswith(".png")]
frame = cv2.imread(os.path.join(img_dir, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter('video.avi', 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(img_dir, image)))

cv2.destroyAllWindows()
video.release()

And repeat the process for other 4 massive galaxies we found earlier. 

This process could be automated, but its execution time will be very long. But here is the code that should work in theory:

In [ ]:
for i in range(5):
    img_dir = parent_dir + '/' + str(ids[i])
    os.chdir(img_dir)                             # Change the working directory to the next folder
    sub = get(subs['results'][i]['url'])          # Get to Subhalo Details page
    simple = get(sub['trees']['sublink_simple'])  # Obtain Simple Sublink Tree data
    sortSimple = sorted(simple['Main'])           # Sort the simple tree data in increasing key value
    
    images = []
    # Get .png file
    for j in range(len(simple['Main'])):
        # Obtain URL that directs to the .png file
        url = 'https://www.tng-project.org/api/TNG100-1/snapshots/'+ str(sortSimple[j][0])+'/subhalos/' + str(sortSimple[j][1])+ '/'
        subhalo = get(url)
        image = get(subhalo['vis']['mergertree_sublink'])
        images.append(image)
        
        
    # Create Video file
    imgs = [img for img in os.listdir(img_dir) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(img_dir, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter('video.avi', 0, 1, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(img_dir, image)))

    cv2.destroyAllWindows()
    video.release()